In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset

import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

from data_loading_code import getData, processUserInput
from model import NeuralNet, LitNet

In [2]:
# get the preprocessed Data
x_train, y_train, x_val, y_val, vocabsize, word_vectorizer = getData("amazon_cells_labelled.txt")

Train data: torch.Size([900, 7277])
Train labels: torch.Size([900])
Validation data: torch.Size([100, 7277])
Validation labels: torch.Size([100])
Vocab size: 7277
Successfully preprocessed the data


In [ ]:
# hyperparameters
EPOCHS = 5
BATCH_SIZE = 10
LEARNING_RATE = 0.001
INPUT_SIZE = vocabsize
HIDDEN_SIZE = 3
NUM_CLASSES = 2 # we only have two classes ?!

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# put data into Dataloaders
train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val, y_val), batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# initialise neural Net
model = NeuralNet(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES)
litModel = LitNet(model)

# initialise logger
logger = TensorBoardLogger("tb_logs", name="chatBotModel")

# intialise Trainer
trainer = L.Trainer(max_epochs=30, 
                    callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
                    accelerator="auto",
                    devices=(1 if torch.cuda.is_available() else None),
                    logger=logger,
                    fast_dev_run=True)

# train the model
trainer.fit(litModel, train_dataloaders=train_loader, val_dataloaders=val_loader)

In [3]:
# chatBot
print(processUserInput("Hallo dies ist ein Test", word_vectorizer))

tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
